# Importing Libraries

In [3]:
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
import os
import pandas as pd
from Mat_Package import Grapher

# Make Giant-Graph

In [4]:
lab_files_path = "./New-Dataset/labels/"
files = os.listdir(lab_files_path)

df = pd.read_csv(lab_files_path+files[0])
G = Grapher.makeGraph(df)
print("Iteration 0")
print("= = = = = = = = = =")

for i in range(1, 100):
    print("Iteration ",i)
    print("= = = = = = = = = =")
    df1 = pd.read_csv(lab_files_path+files[i])
    I_Graph = Grapher.makeGraph(df1)
    G = nx.compose(G, I_Graph)
    


Iteration 0
= = = = = = = = = =
Iteration  1
= = = = = = = = = =
Iteration  2
= = = = = = = = = =
Iteration  3
= = = = = = = = = =
Iteration  4
= = = = = = = = = =
Iteration  5
= = = = = = = = = =
Iteration  6
= = = = = = = = = =
Iteration  7
= = = = = = = = = =
Iteration  8
= = = = = = = = = =
Iteration  9
= = = = = = = = = =
Iteration  10
= = = = = = = = = =
Iteration  11
= = = = = = = = = =
Iteration  12
= = = = = = = = = =
Iteration  13
= = = = = = = = = =
Iteration  14
= = = = = = = = = =
Iteration  15
= = = = = = = = = =
Iteration  16
= = = = = = = = = =
Iteration  17
= = = = = = = = = =
Iteration  18
= = = = = = = = = =
Iteration  19
= = = = = = = = = =
Iteration  20
= = = = = = = = = =
Iteration  21
= = = = = = = = = =
Iteration  22
= = = = = = = = = =
Iteration  23
= = = = = = = = = =
Iteration  24
= = = = = = = = = =
Iteration  25
= = = = = = = = = =
Iteration  26
= = = = = = = = = =
Iteration  27
= = = = = = = = = =
Iteration  28
= = = = = = = = = =
Iteration  29
= = = = = =

# Printing Graph Info

In [5]:
print("No. of Edges in graph are: ", G.number_of_edges())
print("No. of Nodes in graph are: ", G.number_of_nodes())

No. of Edges in graph are:  15543
No. of Nodes in graph are:  3318


# Random-Walk

In [15]:
import random
random.seed(666)

def random_walk(G, start=None, path_length=20, alpha=0, rand=random.Random()):
    '''return a random walk path'''
    if start:
        path = [start]
    else:
        path = [rand.choice(list(G.nodes()))]
    while len(path) < path_length:
        cur = path[-1]
        #find it's neighbors
        if len(G[cur]) > 0:
            if rand.random() >= alpha:
                path.append(rand.choice(list(nx.all_neighbors(G, cur))))
            else:
                path.append(path[0])
        else:
            break
    return path


print(random_walk(G, start=None))


['29', '0', '1.73', '30', '0.99', '30', '40', '3XPB', 'TUB', 'OF', 'YELLOW', '-', '4S', '5', 'OD', '2.2100)', '2.0200]', 'Unit', 'TERATAT,', 'ENAU']


In [16]:
def build_deepwalk_corpus(G, num_paths, rand=random.Random()):
    walks = []
    nodes = list(G.nodes())
    for i in range(num_paths):
        rand.shuffle(nodes)
        for node in nodes:
            walks.append(random_walk(G, start=node))
    return walks


# print(build_deepwalk_corpus(G, num_paths=2))


In [22]:
from gensim.models import Word2Vec
corpus = build_deepwalk_corpus(G, num_paths=20)
model = Word2Vec(corpus, window=2, min_count=1, sg=1)
model.build_vocab(corpus)


# Training Model

In [23]:
model.train(corpus, total_examples=len(corpus),
                epochs=30, report_delay=1)


(38667435, 39816000)

# Checking Word-Similarity

In [25]:
model.wv.most_similar('Date:')

[('#5294961', 0.6670296788215637),
 ('22nd', 0.6404646635055542),
 ('#5367461', 0.6353241801261902),
 ('26/11/2017', 0.633196234703064),
 ('#5438153', 0.6315784454345703),
 ('16/01/2018', 0.6313558220863342),
 ('6/2/2019', 0.6213861703872681),
 ('Issue:', 0.6177234053611755),
 ('30/04/2018', 0.6118603944778442),
 ('14/04/2019', 0.5981958508491516)]

# Making Word-Embeddings

In [27]:
_embeddings = []
all_Text = []

for i in range(100):
    df = pd.read_csv(lab_files_path+files[i])
    Text = df['Object'].to_list()
    
    for T in Text:
        all_Text.append(T)
    
print("Total text is: ", len(all_Text))
print("Making Embeddings..") 

Nodes = G.number_of_nodes()
print(model.wv["Date:"])

# _embeddings = [model.wv[str(n)] for n in range(Nodes)]
# print(_embeddings)

for N in Nodes:
    try:
        _embeddings.append(model.wv[G[N]])
    except:
        print("Node is: ", str(N))

Total text is:  19322
Making Embeddings..
[-0.55012393  0.7139246  -0.3460905  -0.6331017   0.08769065 -0.5641877
  0.79905677  0.8235238   1.1576807  -0.55762637 -0.11934128  0.24390332
  0.20734529 -0.0963482  -0.08149341 -0.3954622   0.74983305 -0.61203766
 -0.7726431  -0.82390666  0.50465566 -0.47784287 -0.3081769   0.18802781
 -0.28980708 -0.04331554 -0.21804406 -0.18795124 -1.3122607  -0.08500629
  0.17828383  0.23447344  0.59734195 -0.05617864  0.38229236 -0.46964502
  0.24714775 -0.35297826 -0.06052864 -0.7316533   0.53589404  0.2144935
 -0.44456     0.7088583   0.34106988 -0.34567946 -1.0070534  -0.31076527
  0.07339371 -0.08959388  0.4815764  -0.8625183  -1.115783   -0.32950434
  0.65789896  0.27299604  0.9021838  -0.25051668  0.13969098 -0.3041152
 -0.25416508 -0.0044085  -0.37462714  0.15621875 -0.15890482  0.9848199
  0.2524382   0.6577557  -0.70230323 -0.2757301  -0.24970122 -0.19398642
  0.44343516  0.3741175  -0.04384673  0.00691444 -0.29309127  0.00925235
 -0.33700606 